# Instructions Provided

In [1]:
# obtains all the reviews for a specific restaurant from Yelp.com.  
# define a variable called yelp_restaurant_url and set it equal to:

#"https://www.yelp.com/biz/j-wata-temaki-bar-san-diego-2?osq=restaurant"
#objective : obtain all the reviews for this restaurant.
#Specifically, you will need to output the reviews to a csv file with the following format
#Each row is a unique review

# Column 1: restaurant name
# Column 2: reviewer’s name
# Column 3: review text

In [2]:
#This is the link to the Yelp restaurant review page for J Wata Temaki Bar in San Diego. 
yelp_restaurant_url = "https://www.yelp.com/biz/j-wata-temaki-bar-san-diego-2?osq=restaurant"

#2nd Page: https://www.yelp.com/biz/j-wata-temaki-bar-san-diego-4?osq=restaurant&start=20
#On 1st Page there are 20 reviews, so on 2nd page though the review # starts from 21, the url has the number 20 in it
#For 1st page this url also works:
# https://www.yelp.com/biz/j-wata-temaki-bar-san-diego-4?osq=restaurant&start=0

# the link should still work even if we change the url to:
# “https://www.yelp.com/biz/greyhouse-coffee-and-supply-west-lafayette-2”

# Headless Browsing

In [3]:
#Headless browsing
import time
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
#choice: define a bunch of user agents, OS types, different browser with different OS
#random browser, with different OS
from random import choice #This library helps pick a random item from a list

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

# These are the usr agents for each of different browsers. Here we are using five, but it can be any number of user agents

In [4]:
class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

myopener = MyOpener()
page=myopener.open(yelp_restaurant_url)

html = page.read().decode('utf-8')

/Users/robin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  after removing the cwd from sys.path.


# Loop all reviews of all pages in a list

In [5]:
#Initiate the list
restaurant_name = []
reviewer_name = []
review_text = []

In [6]:
last_page = False
while last_page == False:
    
    ######################################
    #This script is used as deliminater to deliminate the reviews part
    sub_html1_index = html.find('<script type="application/ld+json">')
    html1 = html[sub_html1_index:]

    end_html1_index = html1.find("</script>")

    #This consolidated html has all the reviews
    html2 = html1[:end_html1_index]
    #html2

    #Inner while loop
    while html2.find('\"author":') != -1: 

        #Part1: Find the review text

        #We are running a while loop because we want to get everything in the page
        #The review starts after description
        review_start_index = html2.find('\"description":')
        #review_start_index

        #16: the digits in description and quotes and 1 space
        html_review = html2[review_start_index+16:]
        #html_review

        #the review ends when their is a name of author in the code
        review_end_index = html_review.find('"author":')

        #review end 3 places before the author word starts
        review = html_review[:review_end_index-3]
        #review    

        #add the review in the review list
        review_text.append(review) 
        #review_text

        #Part2: Find the name of reviewer
        #The remaining is the html from the end of the review till the end of the html
        remaining = html_review[review_end_index:]

        reviewer_name_start_index = 11

        #Just at the end of the author name, there is a paranthesis
        reviewer_name_end_index = remaining.find('"}')

        reviewer = remaining[reviewer_name_start_index: reviewer_name_end_index]

        #add the name of the reviewer in the reviewer list
        reviewer_name.append(reviewer)

        #Part3: Truncate the html code
        html2 = remaining[reviewer_name_end_index+1: ] 

        #After coming out of the while we can find the name of the restraunt from the remaining html2 code
        #This is kept inside the while loop itself so that we can get the # of restraunts same as # of reviews
        res_name_index = html2.find('\"name":')
        html3 = html2[res_name_index:]

        res_name_index_end = html3.find(', "address":')
        restaurant = html3[9:res_name_index_end-1]
        restaurant_name.append(restaurant)

    ##################################################
    
    guess_next_index = html.find('<script type="application/ld+json">')
    html_guess_next = html[:guess_next_index]
    next_check = html_guess_next.find('<link rel="next" href="')

    if next_check == -1:
            last_page = True
    
    else:
        link_html = html_guess_next[next_check:]
        link_start_index = link_html.find("https:")
        #link_start_index

        link_end_index = link_html.find("\" />\n\n")
        #link_end_index

        #Next link to go through
        #Name the link as html so that the inner while loop will iterate
        yelp_restaurant_url = link_html[link_start_index:link_end_index]
        
        class MyOpener(FancyURLopener, object):
            version = choice(user_agents)

        myopener = MyOpener()
        page=myopener.open(yelp_restaurant_url)

        html = page.read().decode('utf-8')      
        last_page = False

/Users/robin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods


In [7]:
#Its correct because 1 review is in chinese
#len(reviewer_name) = number of reviews = 259
#len(review_text) = number of reviews = 259
#len(restaurant_name) : 259 = number of reviews

# Cleaning the Reviews

In [8]:
#Place a for loop so as to replace all the \\n & \t & \n with " " (a space)
for x in range(0, len(review_text)):
    review_text[x] = review_text[x].replace("\\n", " ")
    review_text[x] = review_text[x].replace("\n", " ")
    review_text[x] = review_text[x].replace("\t", " ")

# Exporting the data to .csv file

In [9]:
#Create a write access enabled txt file named Mutual_Fund_CIK
Output_File = open("Files_Directory/Edgar/Yelp_Reviews.csv", "w")

#Add the heading
Output_File.write("Restaurant name\tReviewer’s name\tReview text\n")

#for all the CIKs and corresponding Fund Names
for x in range(0, len(reviewer_name)):
    Output_File.write(restaurant_name[x] + '\t' + reviewer_name[x] + '\t' + review_text[x] + '\t\n')

Output_File.close()